# Use scikit-learn to predict diabetes progression with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of external machine learning models in Watson Machine Learning Service. This notebook introduces `cURL` calls for publishing, deploying (Web Service) and scoring ML scikit model.

Some familiarity with cURL is helpful. This notebook uses cURL examples.

This example was made based on `diabetes` dataset and trained scikit ML model that could be found inside the same repository under as follows: 
- `/data/diabetes/`
- `/models/scikit/diabetes/`


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning repository, deployment and scoring.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)  
2. [WML Repository](#wml_repository)   
3. [Model Deployment and Scoring](#deploy_and_score)   
4. [Persist new version of the model](#update_model)
5. [Redeploy and score new version of the model](#redeploy)
6. [Cleaning](#cleaning)  
7. [Summary](#summary)  

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html" target="_blank" rel="noopener no referrer">here</a>).

You can find your COS credentials in COS instance dashboard under the **Service credentials** tab.
Go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information.

Authenticate the Watson Machine Learning service on IBM Cloud.

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.  

**NOTE:** You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.  



In [ ]:
from json import loads
from IPython.display import JSON

In [ ]:
%env API_KEY=...
%env WML_ENDPOINT_URL=...
%env WML_INSTANCE_CRN="fill out only if you want to create a new space"
%env WML_INSTANCE_NAME=...

%env COS_CRN="fill out only if you want to create a new space"
%env COS_ENDPOINT=...
%env COS_BUCKET=...
%env COS_ACCESS_KEY_ID=...
%env COS_SECRET_ACCESS_KEY=...
%env COS_API_KEY=...

%env SPACE_ID="fill out only if you have space already created"

%env DATAPLATFORM_URL=https://api.dataplatform.cloud.ibm.com   
%env AUTH_ENDPOINT=https://iam.cloud.ibm.com/oidc/token

<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [ ]:
%%bash --out token

curl -sk -X POST \
    --header "Content-Type: application/x-www-form-urlencoded" \
    --header "Accept: application/json" \
    --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
    --data-urlencode "apikey=$API_KEY" \
    "$AUTH_ENDPOINT" \
    | cut -d '"' -f 4

In [ ]:
%env TOKEN=$token

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

---

<a id="wml_repository"></a>
## 2. Manage WML Repository

In this section you will learn how to upload your ML model, list your models stored in repository, delete your model and update it.

<a id="model_storing"></a>
### Model storing
Store information about your model to WML repository.

In [ ]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "This is description","type": "scikit-learn_1.0", "software_spec": {"name": "runtime-22.1-py3.9"}}'
echo $MODEL_PAYLOAD

In [ ]:
JSON(loads(model_payload))

In [ ]:
%env MODEL_PAYLOAD=$model_payload

In [ ]:
%%bash --out model_id -s "$model_payload"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [ ]:
%env MODEL_ID=$model_id

### Create model revision for further update

In [ ]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Initial model."}'
echo $REVISION_PAYLOAD

In [ ]:
JSON(loads(revision_payload))

In [ ]:
%env REVISION_PAYLOAD=$revision_payload

In [ ]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?version=2020-08-01" \
    | python -m json.tool

<a id="content_upload"></a>
### Model content upload
Now you need to upload your model content into the WML repository.

In [ ]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/scikit/diabetes/model/diabetes_model.tar.gz \
    -O diabetes_model.tar.gz

In [ ]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@diabetes_model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | grep "attachment_id" | awk -F '"' '{ print $4 }'

In [ ]:
%env ATTACHMENT_ID=$attachment_id

<a id="model_download"></a>
### Download model
If you want to download your saved model, please make the following call.

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [ ]:
!ls -l model.tar.gz

<a id="deploy_and_score"></a>
## 3. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

<a id="deployment_creation"></a>
### Deployment creation
This example uses scikit-learn model deployment and `S` hardware specification.

In [ ]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD

In [ ]:
JSON(loads(deployment_payload))

In [ ]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

In [ ]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [ ]:
%env DEPLOYMENT_ID=$deployment_id

<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="webservice_score"></a>
### Scoring of a webservice
If you want to make a `score` call on your deployment, please follow a below method:

In [ ]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}'
echo $SCORING_PAYLOAD

In [ ]:
JSON(loads(scoring_payload))

In [ ]:
%env SCORING_PAYLOAD=$scoring_payload

In [ ]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD"\
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

<a id="deployments_list"></a>
### Listing all deployments

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="update_model"></a>
## 4. Persist new version of the model

In this section, you'll learn how to store new version of your model in Watson Machine Learning repository.

### Model update
Below you can find how ML model can be updated with new version on WML repository.

List model revisions.

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

### Create second model revision

In [ ]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updated model."}'
echo $REVISION_PAYLOAD

In [ ]:
JSON(loads(revision_payload))

In [ ]:
%env REVISION_PAYLOAD=$revision_payload

In [ ]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?version=2020-08-01" \
    | python -m json.tool

### Update model metadata  
  
For example update  model name or description

In [ ]:
%%bash --out update_payload

echo '[{"op": "add", "path": "/name", "value": "updated scikit model"}]'

In [ ]:
%env UPDATE_PAYLOAD=$update_payload

In [ ]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$UPDATE_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

### Upload new model content

In [ ]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/scikit/diabetes/model/new_diabetes_model.tar.gz \
    -O new_diabetes_model.tar.gz

In [ ]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@new_diabetes_model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | python -m json.tool

#### List model revisions to see a new one just created

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

Now we have updated model content and model name in repository.

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="redeploy"></a>
## 5. Redeploy and score new version of the model

Below you can see how deployment can be updated with new version of the model without any change for scoring url.

In [ ]:
%%bash --out redeploy_payload

echo '[{"op": "replace", "path": "/asset", "value": {"id": "'"$MODEL_ID"'"}}]'

In [ ]:
%env REDEPLOY_PAYLOAD=$redeploy_payload

In [ ]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REDEPLOY_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

### Score updated webservice

In [ ]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

%%bash

curl -sk -X DELETE \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01"

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

%%bash

curl -sk -X DELETE \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01"

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a scikit-learn ML model in WML. 


### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.